In [1]:
from sklearn import linear_model
import sklearn
import json
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy
import random
import gzip
import math
import string
import scipy
from scipy import sparse
from implicit import bpr
import tensorflow as tf
from collections import defaultdict

import gzip
from collections import defaultdict
import re
import datetime

from typing import List


c:\Users\traip\OneDrive\Desktop\CSEstuff\CSE_158\assignment_2\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [ ]:
steam_reviews_data = []
user_reviews_data = []
user_libraries_data = []
bundle_data = []
games_data = []

with gzip.open("steam_reviews.json.gz") as file:
    """

    Gives a list of user and all their reviews, within the review it shows how if the review was funny, the date, how helpful, the recommendation, and review 
    """
    for l in file:
        d = eval(l)
        steam_reviews_data.append(d)
        if len(steam_reviews_data) >= 60000:
            break

with gzip.open("australian_user_reviews.json.gz") as file:
    """
    {user_id: int, user_url: str, reviews: list[{funny: bool, posted: str, last_edited: str, item_id: int, helpful: int, recommend: Bool, review: str}]}
    
    Gives a list of user and all their reviews, within the review it shows how if the review was funny, the date, how helpful, the recommendation, and review 
    """
    for l in file:
        d = eval(l)
        user_reviews_data.append(d)
        if len(user_reviews_data) >= 60000:
            break

with gzip.open("australian_users_items.json.gz") as file:
    """
    {user_id: int, items_count: int, steam_id: int, user_url: str, items: list[{item_id: int, item_name: str, playtime_forever: int, playtime_2weeks: 0}]}
    
    Contains data of existing games user have in their library and their playtime
    
    """
    for l in file:
        d = eval(l)
        user_libraries_data.append(d)
        if len(user_libraries_data) >= 60000:
            break

with gzip.open("bundle_data.json.gz") as file:
    for l in file:
        d = eval(l)
        bundle_data.append(d)
        if len(bundle_data) >= 60000:
            break

with gzip.open("steam_games.json.gz") as file:
    """
    {'publisher': str, u'genres': list[str], u'app_name': str, u'title': str, u'url': str, u'release_date': str, u'tags': list[str], u'discount_price': float, u'reviews_url': str, u'specs': list[str], u'price': float, u'early_access': bool, u'id': str, u'developer': str" }

    Gives all information about the game suchas genre/tags, name, release-data, price/discount_price, review_url, early access
    :ISSUE: Release date isn't always of format "YYYY-mm-dd"
    """
    for l in file:
        d = eval(l)
        games_data.append(d)
        if len(games_data) >= 60000:
            break


In [ ]:
# Print statements
print(f"steam reivew: {steam_reviews_data[0]} \n")
print(f"user reivew: {user_reviews_data[0]} \n")
print(f"user library: {user_libraries_data[12]} \n")
print(f"bundle data: {bundle_data[0]} \n")
print(f"game: {games_data[0]} \n")


In [ ]:
print(f"steam review len: {len(steam_reviews_data)} \n")
print(f"user review len: {len(user_reviews_data)} \n")
print(f"user library: {len(user_libraries_data)} \n")
print(f"bundle: {len(bundle_data)} \n")
print(f"game len: {len(games_data)} \n")


In [ ]:
# get data structure
"List of genres"
genre_list = set()
"Set of users and their reviews"
user_to_reviews = defaultdict(list)
for user in user_reviews_data:
    user_to_reviews[user["user_id"]] = (user['reviews'])
"List of user, and their amount of games in their library, and their number of recommended reviews"
user_rec_games = defaultdict(list)
for game in games_data:
    try:
        for genre in game['genres']:
            genre_list.add(genre)
    except:
        for genre in game['tags']:
            genre_list.add(genre)
    finally:
        continue

for user in user_libraries_data:
    total_num = user['items_count']
    u_id = user['user_id']
    if u_id in user_rec_games:
        continue
    if u_id in user_to_reviews:
        rec_count = 0
        for rev in user_to_reviews[u_id]:
            if rev['recommend']:
                rec_count += 1
        user_rec_games[u_id].append((total_num, rec_count))
    else:
        user_rec_games[u_id].append((total_num, 0))


## Plot 1


In [ ]:
# # recommendation vs # games user have
x_num_games = []
y_num_rec = []
for l in user_rec_games.values():
    x_num_games.append(l[0][0])
    y_num_rec.append(l[0][1])

plt.plot(y_num_rec, x_num_games, 'bo', label="# Games Owned vs # Rec.",)
plt.show()


## Plot 2